<a href="https://colab.research.google.com/github/khurramjanjua5/deep-learning-projects/blob/main/Dogs_vs_Cats_using_RESNET_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle


ls: cannot access '/root/.kaggle/': Not a directory


In [2]:
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d salader/dogs-vs-cats

 98% 1.04G/1.06G [00:11<00:00, 194MB/s]
100% 1.06G/1.06G [00:11<00:00, 102MB/s]


In [4]:
import zipfile

# Path to the zip file
zip_file_path = '/content/dogs-vs-cats.zip'

# Directory to extract the contents
extract_to_path = '/content/dogs-vs-cats'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Dataset extracted successfully.")


Dataset extracted successfully.


In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model

In [6]:
# Load the pre-trained ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

94765736/94765736 [==============================] - 1s 0us/step


In [14]:
#generators to divide the data into batches with their labels so that data can be processed one by one. Generators are commonly used when working with large datasets that cannot fit entirely into memory.
train_ds=keras.utils.image_dataset_from_directory(
    directory='/content/dogs-vs-cats/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)

validation_ds=keras.utils.image_dataset_from_directory(
    directory='/content/dogs-vs-cats/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256,256)
)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [15]:

#Normalize
def process(image,label):
  image=tf.cast(image/255,tf.float32)
  return image,label

  train_ds=train_ds.map(process)
  validation_ds=validation_ds(process)

In [7]:
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Add your custom classifier on top of the base ResNet-50 model
x = layers.Flatten()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)


In [9]:
# Combine the base model with the custom classifier
model = Model(inputs=base_model.input, outputs=output)


In [17]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [18]:
history=model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
625/625 [==============================] - 107s 156ms/step - loss: 0.4073 - accuracy: 0.9581 - val_loss: 0.1028 - val_accuracy: 0.9738
Epoch 2/10
625/625 [==============================] - 99s 157ms/step - loss: 0.0828 - accuracy: 0.9737 - val_loss: 0.0701 - val_accuracy: 0.9774
Epoch 3/10
625/625 [==============================] - 98s 157ms/step - loss: 0.0647 - accuracy: 0.9785 - val_loss: 0.0580 - val_accuracy: 0.9802
Epoch 4/10
625/625 [==============================] - 96s 153ms/step - loss: 0.0500 - accuracy: 0.9808 - val_loss: 0.0759 - val_accuracy: 0.9790
Epoch 5/10
625/625 [==============================] - 96s 153ms/step - loss: 0.0479 - accuracy: 0.9837 - val_loss: 0.0740 - val_accuracy: 0.9794
Epoch 6/10
625/625 [==============================] - 97s 154ms/step - loss: 0.0477 - accuracy: 0.9841 - val_loss: 0.0844 - val_accuracy: 0.9812
Epoch 7/10
625/625 [==============================] - 96s 153ms/step - loss: 0.0391 - accuracy: 0.9868 - val_loss: 0.0689 - val_a